* download front image

In [26]:
# pip install boto3
import boto3
from botocore.exceptions import ClientError

BUCKET = "amazon-bodym"
KEY = "testA/mask/01b7f357ef9000d55e74891773d42d8a.png"  # adjust if your prefix differs
OUT_PATH = "front.png"  # saves as "front"

# Option A (common): use default AWS creds (env vars, ~/.aws/credentials, IAM role, etc.)
s3 = boto3.client("s3")

# Option B: use a named AWS profile instead (uncomment)
# session = boto3.Session(profile_name="default")  # e.g., "myprofile"
# s3 = session.client("s3")

try:
    s3.download_file(BUCKET, KEY, OUT_PATH)
    print(f"Downloaded s3://{BUCKET}/{KEY} -> {OUT_PATH}")
except ClientError as e:
    print("Download failed:", e)
    raise


Downloaded s3://amazon-bodym/testA/mask/01b7f357ef9000d55e74891773d42d8a.png -> front.png


* download side image

In [27]:
# pip install boto3
import boto3
from botocore.exceptions import ClientError

BUCKET = "amazon-bodym"
KEY = "testA/mask_left/01b7f357ef9000d55e74891773d42d8a.png"  # adjust if your prefix differs
OUT_PATH = "side.png"  # saves as "front"

# Option A (common): use default AWS creds (env vars, ~/.aws/credentials, IAM role, etc.)
s3 = boto3.client("s3")

# Option B: use a named AWS profile instead (uncomment)
# session = boto3.Session(profile_name="default")  # e.g., "myprofile"
# s3 = session.client("s3")

try:
    s3.download_file(BUCKET, KEY, OUT_PATH)
    print(f"Downloaded s3://{BUCKET}/{KEY} -> {OUT_PATH}")
except ClientError as e:
    print("Download failed:", e)
    raise


Downloaded s3://amazon-bodym/testA/mask_left/01b7f357ef9000d55e74891773d42d8a.png -> side.png


* extract the subject_id related to photo_id

In [28]:
# pip install boto3 pandas
import boto3
import pandas as pd
from io import BytesIO

BUCKET = "amazon-bodym"
CSV_KEY = "testA/subject_to_photo_map.csv"  # from your screenshot
PHOTO_ID = "01b7f357ef9000d55e74891773d42d8a"

s3 = boto3.client("s3")

# Load CSV from S3 into a DataFrame
obj = s3.get_object(Bucket=BUCKET, Key=CSV_KEY)
df = pd.read_csv(BytesIO(obj["Body"].read()))

# Try common column names (adjust if your CSV uses different headers)
# Expected: columns like ["subject_id", "photo_id"]
matches = df[df["photo_id"].astype(str).str.replace(".png", "", regex=False) == PHOTO_ID]

if matches.empty:
    print("No subject_id found for that photo_id.")
else:
    # Print unique subject_ids (in case there are duplicates)
    subject_ids = matches["subject_id"].unique().tolist()
    print("subject_id(s):", subject_ids)


subject_id(s): ['a0iE7PPHJxEy8GXFpMtwTdy5QNfOL8F8oB1XDoNi7jI']


In [29]:
# pip install boto3 pandas
import boto3
import pandas as pd
from io import BytesIO

BUCKET = "amazon-bodym"
HWG_KEY = "testA/hwg_metadata.csv"  # from your screenshot
SUBJECT_ID = "a0iE7PPHJxEy8GXFpMtwTdy5QNfOL8F8oB1XDoNi7jI"

s3 = boto3.client("s3")

# Load hwg_metadata.csv from S3
obj = s3.get_object(Bucket=BUCKET, Key=HWG_KEY)
df = pd.read_csv(BytesIO(obj["Body"].read()))

# Find the subject row (try common subject id column names)
subject_col = None
for c in ["subject_id", "subjectId", "subject", "id"]:
    if c in df.columns:
        subject_col = c
        break
if subject_col is None:
    raise KeyError(f"Couldn't find a subject id column. Columns are: {list(df.columns)}")

row = df[df[subject_col].astype(str) == SUBJECT_ID]
if row.empty:
    print("No row found for that subject_id.")
    print("Available columns:", list(df.columns))
else:
    # Find height and gender columns (common variants)
    def pick_col(candidates):
        for c in candidates:
            if c in df.columns:
                return c
        return None

    height_col = pick_col(["height_cm", "heightCm", "height", "Height_cm", "HEIGHT_CM"])
    gender_col = pick_col(["gender", "Gender", "sex", "Sex"])

    if height_col is None or gender_col is None:
        print("Couldn't auto-detect height/gender columns.")
        print("Available columns:", list(df.columns))
    else:
        height_cm = row.iloc[0][height_col]
        gender = row.iloc[0][gender_col]
        print(f"subject_id: {SUBJECT_ID}")
        print(f"height_cm: {height_cm}")
        print(f"gender: {gender}")


subject_id: a0iE7PPHJxEy8GXFpMtwTdy5QNfOL8F8oB1XDoNi7jI
height_cm: 166.2
gender: female
